In [4]:
!pip install -U langchain openai tiktoken



In [5]:
import inspect

from langchain import OpenAI
from langchain.chains import LLMChain, ConversationChain
from langchain.chains.conversation.memory import (ConversationBufferMemory,
                                                  ConversationSummaryMemory,
                                                  ConversationBufferWindowMemory,
                                                  ConversationKGMemory)
from langchain.callbacks import get_openai_callback
import tiktoken

In [6]:
memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

In [7]:
memory.load_memory_variables({})

{'history': "Human: hi!\nAI: what's up?"}

In [8]:
memory = ConversationBufferMemory(memory_key="chat_history")
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

In [9]:
memory.load_memory_variables({})

{'chat_history': "Human: hi!\nAI: what's up?"}

In [10]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory.chat_memory.add_user_message("hi!")
memory.chat_memory.add_ai_message("what's up?")

In [11]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='hi!'),
  AIMessage(content="what's up?")]}

In [12]:
import os
os.environ["OPENAI_API_KEY"] = "your api key"

In [13]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory


llm = OpenAI(temperature=0)
# Notice that "chat_history" is present in the prompt template
template = """You are a nice chatbot having a conversation with a human.

Previous conversation:
{chat_history}

New human question: {question}
Response:"""
prompt = PromptTemplate.from_template(template)
# Notice that we need to align the `memory_key`
memory = ConversationBufferMemory(memory_key="chat_history")
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

In [14]:
# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
conversation({"question": "hi"})



> Entering new LLMChain chain...
Prompt after formatting:
You are a nice chatbot having a conversation with a human.

Previous conversation:


New human question: hi
Response:

> Finished chain.


{'question': 'hi',
 'chat_history': '',
 'text': ' Hello! How are you doing today?'}

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory


llm = ChatOpenAI()
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a nice chatbot having a conversation with a human."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)
# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name.
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

In [16]:
# Notice that we just pass in the `question` variables - `chat_history` gets populated by memory
conversation({"question": "hi"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a nice chatbot having a conversation with a human.
Human: hi

> Finished chain.


{'question': 'hi',
 'chat_history': [HumanMessage(content='hi'),
  AIMessage(content='Hello! How can I assist you today?')],
 'text': 'Hello! How can I assist you today?'}

In [17]:
from langchain.chains.conversation.memory import ConversationBufferMemory
conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

In [18]:
conversation = ConversationChain(
    llm=llm,
)

In [19]:
print(inspect.getsource(conversation._call), inspect.getsource(conversation.apply))

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        response = self.generate([inputs], run_manager=run_manager)
        return self.create_outputs(response)[0]
     def apply(
        self, input_list: List[Dict[str, Any]], callbacks: Callbacks = None
    ) -> List[Dict[str, str]]:
        """Utilize the LLM generate method for speed gains."""
        callback_manager = CallbackManager.configure(
            callbacks, self.callbacks, self.verbose
        )
        run_manager = callback_manager.on_chain_start(
            dumpd(self),
            {"input_list": input_list},
        )
        try:
            response = self.generate(input_list, run_manager=run_manager)
        except BaseException as e:
            run_manager.on_chain_error(e)
            raise e
        outputs = self.create_outputs(response)
        run_manager.on_chain_end({"outputs": outputs})
        retur

In [20]:
print(inspect.getsource(LLMChain._call), inspect.getsource(LLMChain.apply))

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        response = self.generate([inputs], run_manager=run_manager)
        return self.create_outputs(response)[0]
     def apply(
        self, input_list: List[Dict[str, Any]], callbacks: Callbacks = None
    ) -> List[Dict[str, str]]:
        """Utilize the LLM generate method for speed gains."""
        callback_manager = CallbackManager.configure(
            callbacks, self.callbacks, self.verbose
        )
        run_manager = callback_manager.on_chain_start(
            dumpd(self),
            {"input_list": input_list},
        )
        try:
            response = self.generate(input_list, run_manager=run_manager)
        except BaseException as e:
            run_manager.on_chain_error(e)
            raise e
        outputs = self.create_outputs(response)
        run_manager.on_chain_end({"outputs": outputs})
        retur

In [21]:
conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

In [22]:
conversation_buf("Good morning AI!")

{'input': 'Good morning AI!',
 'history': '',
 'response': 'Good morning! How can I assist you today?'}

In [23]:
conversation_buf("Can you please tell me your name AI?")

{'input': 'Can you please tell me your name AI?',
 'history': 'Human: Good morning AI!\nAI: Good morning! How can I assist you today?',
 'response': 'Of course! My name is OpenAI, or GPT-3 for short. How can I help you today?'}

In [24]:
def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens')

    return result

In [25]:
count_tokens(
    conversation_buf,
    "This book introduces the concept of a Big Hairy Audacious Goal (BHAG), a long-term, ambitious,\
     and inspiring goal that can motivate and align the efforts of employees and stakeholders."
)


Spent a total of 181 tokens


'That sounds interesting! Would you like me to provide more information about the concept of BHAGs or any specific examples?'

In [26]:
count_tokens(
    conversation_buf,
    "The BHAG can help an organization focus on a long-term goal, create a sense of urgency, and attract and retain top talent. The book encourages readers to embrace their BHAGs and galvanize their organizations towards success."
)

Spent a total of 354 tokens


"That's great to hear! BHAGs can indeed be a powerful tool for organizations. By setting a long-term and ambitious goal, companies can create a sense of urgency and inspire their employees to work towards a common objective. This can lead to increased motivation and productivity within the organization. BHAGs can also attract and retain top talent as they provide a clear and compelling vision for the future. It's wonderful that the book encourages readers to embrace their BHAGs and use them to galvanize their organizations towards success. Is there anything else you would like to know about BHAGs or any other topic?"

In [27]:
count_tokens(
    conversation_buf,
    "What concept this book introduces?"
)

Spent a total of 381 tokens


'This book introduces the concept of a Big Hairy Audacious Goal (BHAG).'

In [28]:
print(conversation_buf.memory.buffer)

Human: Good morning AI!
AI: Good morning! How can I assist you today?
Human: Can you please tell me your name AI?
AI: Of course! My name is OpenAI, or GPT-3 for short. How can I help you today?
Human: This book introduces the concept of a Big Hairy Audacious Goal (BHAG), a long-term, ambitious,     and inspiring goal that can motivate and align the efforts of employees and stakeholders.
AI: That sounds interesting! Would you like me to provide more information about the concept of BHAGs or any specific examples?
Human: The BHAG can help an organization focus on a long-term goal, create a sense of urgency, and attract and retain top talent. The book encourages readers to embrace their BHAGs and galvanize their organizations towards success.
AI: That's great to hear! BHAGs can indeed be a powerful tool for organizations. By setting a long-term and ambitious goal, companies can create a sense of urgency and inspire their employees to work towards a common objective. This can lead to incre

In [29]:
print(conversation_buf.memory.buffer)

Human: Good morning AI!
AI: Good morning! How can I assist you today?
Human: Can you please tell me your name AI?
AI: Of course! My name is OpenAI, or GPT-3 for short. How can I help you today?
Human: This book introduces the concept of a Big Hairy Audacious Goal (BHAG), a long-term, ambitious,     and inspiring goal that can motivate and align the efforts of employees and stakeholders.
AI: That sounds interesting! Would you like me to provide more information about the concept of BHAGs or any specific examples?
Human: The BHAG can help an organization focus on a long-term goal, create a sense of urgency, and attract and retain top talent. The book encourages readers to embrace their BHAGs and galvanize their organizations towards success.
AI: That's great to hear! BHAGs can indeed be a powerful tool for organizations. By setting a long-term and ambitious goal, companies can create a sense of urgency and inspire their employees to work towards a common objective. This can lead to incre

In [30]:
from langchain.chains.conversation.memory import ConversationSummaryMemory

conversation_sum = ConversationChain(
	llm=llm,
	memory=ConversationSummaryMemory(llm=llm)
)


In [31]:
print(conversation_sum.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


In [32]:
# without count_tokens we'd call `conversation_sum("Can you please tell me your name AI?!")`
# but let's keep track of our tokens:
count_tokens(
    conversation_sum,
    "Can you please tell me your name AI?"
)

Spent a total of 288 tokens


'Of course! My name is OpenAI.'

In [33]:
count_tokens(
    conversation_sum,
    "This book introduces the concept of a Big Hairy Audacious Goal (BHAG), a long-term, ambitious,\
     and inspiring goal that can motivate and align the efforts of employees and stakeholders."
)

Spent a total of 663 tokens


"That sounds like a fascinating concept! A Big Hairy Audacious Goal (BHAG) is a powerful tool for organizations to set long-term objectives that are ambitious and inspiring. By having a BHAG, employees and stakeholders can be motivated and aligned towards a common purpose, driving innovation and progress. It's a great way to keep everyone focused and working towards a shared vision."

In [34]:
count_tokens(
    conversation_sum,
    "The BHAG can help an organization focus on a long-term goal, create a sense of urgency, and attract and retain top talent. The book encourages readers to embrace their BHAGs and galvanize their organizations towards success."
)


Spent a total of 886 tokens


'That sounds like a great strategy! By setting a Big Hairy Audacious Goal (BHAG), organizations can indeed create a sense of urgency and align their employees towards a common objective. It can also help attract and retain top talent, as people are often motivated by the opportunity to work towards ambitious and inspiring goals. The book you mentioned seems to provide valuable insights on how to embrace BHAGs and drive success within organizations.'

In [35]:
count_tokens(
    conversation_sum,
    "What concept this book introduces?"
)

Spent a total of 755 tokens


'The book introduces the concept of a Big Hairy Audacious Goal (BHAG).'

In [36]:
count_tokens(
    conversation_sum,
    "the book encourages readers of what?"
)

Spent a total of 882 tokens


'The book encourages readers to embrace the concept of a Big Hairy Audacious Goal (BHAG) and provides insights on how to drive success within organizations by setting and pursuing these ambitious goals.'

In [37]:
print(conversation_sum.memory.buffer)

The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. The human asks the AI for its name, and the AI responds that its name is OpenAI. The human introduces the concept of a Big Hairy Audacious Goal (BHAG), explaining its benefits. The AI finds the concept fascinating and explains further how a BHAG can help organizations. The human mentions that embracing BHAGs can create a sense of urgency, attract top talent, and drive success. The AI agrees and adds that setting a BHAG can align employees, motivate them, and provide a shared vision. The AI also acknowledges the value of the book mentioned in providing insights on embracing BHAGs and driving success within organizations. The human asks what concept this book introduces, and the AI responds that the book encourages readers to embrace the concept of a Big Hairy Audacious Goal (BHAG) and provides insights on how to d

In [38]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

conversation = ConversationChain(
	llm=llm,
	memory=ConversationBufferWindowMemory(k=1)
)

In [39]:
conversation_bufw = ConversationChain(
    llm=llm,
    memory=ConversationBufferWindowMemory(k=1)
)

In [40]:
count_tokens(
    conversation_bufw,
    "Good morning AI!"
)

Spent a total of 77 tokens


'Good morning! How can I assist you today?'

In [41]:
count_tokens(
    conversation_bufw,
    "I want to know more about Singularity, what it is and when it is coming."
)


Spent a total of 301 tokens


'Singularity refers to a hypothetical future point in time when technological growth becomes uncontrollable and irreversible, resulting in unforeseeable changes to human civilization. It is often associated with the emergence of superintelligent machines that surpass human intelligence in every aspect. The concept was popularized by mathematician and computer scientist Vernor Vinge in the 1990s and further explored by futurist Ray Kurzweil.\n\nAs for when Singularity might occur, it is difficult to predict with certainty. Some experts believe it could happen within the next few decades, while others argue that it is still far off in the future. The development and arrival of superintelligent AI systems are dependent on various factors, including advancements in technology, algorithms, and computational power.\n\nIt is worth noting that the concept of Singularity is highly debated, and there are different opinions regarding its feasibility and potential impact. Some see it as a potentia

In [42]:
count_tokens(
    conversation_bufw,
    "What I want to know?"
)

Spent a total of 323 tokens


'Please let me know what specific information you would like to know about Singularity, and I will do my best to provide you with the requested details.'

In [43]:
count_tokens(
    conversation_bufw,
    "Give me the technology behind it"
)


Spent a total of 449 tokens


'The concept of Singularity refers to a hypothetical point in the future where artificial intelligence (AI) surpasses human intelligence, leading to rapid technological advancements that are beyond our current comprehension. While the idea of Singularity encompasses various aspects, there are several key technologies that are often associated with its potential realization.\n\nOne of the fundamental technologies behind Singularity is artificial general intelligence (AGI). AGI refers to highly autonomous systems that possess the ability to outperform humans in most economically valuable work. It involves creating AI systems that can understand, learn, and apply knowledge across a wide range of tasks and domains. AGI is an essential component for achieving Singularity since it requires machines that are capable of surpassing human intellectual capabilities.\n\nAnother technology that plays a crucial role in Singularity is machine learning. Machine learning involves developing algorithms 

In [44]:
count_tokens(
    conversation_bufw,
    "What I want to know?"
)

Spent a total of 468 tokens


'I apologize, but I do not have the ability to know what specific information you want to know without you telling me. Please provide me with your question or topic of interest, and I will do my best to provide you with the relevant information.'

In [45]:
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory

In [46]:
conversation_sum_bufw = ConversationChain(
    llm=llm,
    memory=ConversationSummaryBufferMemory(
        llm=llm,
        max_token_limit=650
    ),
)


In [47]:
print(conversation_sum_bufw.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:
